Методы отбора признаков
- RFE  
sklearn.feature_selection.RFE
- PCA  
- Tree-based  
sklearn.ensemble.ExtraTreesClassifier
- SVM + SelectFromModel  
sklearn.svm.SVC
sklearn.feature_selection.SelectFromModel


roc auc score надо посчитать

https://www.kaggle.com/competitions/playground-series-s3e22/data?select=train.csv

In [7]:
import pandas as pd
import numpy as np 
pd.set_option('display.max_columns', None)

In [29]:
path = 'data/train.csv'
hourse_data = pd.read_csv(path, index_col=0)

In [30]:
hourse_data.head()

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,pain,peristalsis,abdominal_distention,nasogastric_tube,nasogastric_reflux,nasogastric_reflux_ph,rectal_exam_feces,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data,outcome
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,yes,adult,530001,38.1,132.0,24.0,cool,reduced,dark_cyanotic,more_3_sec,depressed,absent,slight,slight,less_1_liter,6.5,decreased,distend_small,57.0,8.5,serosanguious,3.4,yes,2209,0,0,no,died
1,yes,adult,533836,37.5,88.0,12.0,cool,normal,pale_cyanotic,more_3_sec,mild_pain,absent,moderate,none,more_1_liter,2.0,absent,distend_small,33.0,64.0,serosanguious,2.0,yes,2208,0,0,no,euthanized
2,yes,adult,529812,38.3,120.0,28.0,cool,reduced,pale_pink,less_3_sec,extreme_pain,hypomotile,moderate,slight,none,3.5,NaN,distend_large,37.0,6.4,serosanguious,3.4,yes,5124,0,0,no,lived
3,yes,adult,5262541,37.1,72.0,30.0,cold,reduced,pale_pink,more_3_sec,mild_pain,hypomotile,moderate,slight,more_1_liter,2.0,decreased,distend_small,53.0,7.0,cloudy,3.9,yes,2208,0,0,yes,lived
4,no,adult,5299629,38.0,52.0,48.0,normal,normal,normal_pink,less_3_sec,alert,hypomotile,none,slight,less_1_liter,7.0,normal,normal,47.0,7.3,cloudy,2.6,no,0,0,0,yes,lived


- surgery
  - Yes, it had surgery
  - It was treated without surgery
- Age
  - Adult horse
  - Young (< 6 months)
- Hospital Number
  - numeric id
  - the case number assigned to the horse (may not be unique if the horse is treated > 1 time)
- rectal temperature
  - linear
  - in degrees celsius
  - An elevated temp may occur due to infection
  - temperature may be reduced when the animal is in late shock
  - normal temp is 37.8
  - this parameter will usually change as the problem progresses, eg. may start out normal, then become elevated because of the lesion, passing back through the normal range as the horse goes into shock 
- pulse
  - linear
  - the heart rate in beats per minute
  - is a reflection of the heart condition: 30 -40 is normal for adults
  - rare to have a lower than normal rate although athletic horses may have a rate of 20-25 
  - animals with painful lesions or suffering from circulatory shock may have an elevated heart rate 
- respiratory rate 
  - linear
  - normal rate is 8 to 10
  - usefulness is doubtful due to the great fluctuations
- temperature of extremities (cool to cold extremities indicate possible shock)
  - Normal 
  - Warm 
  - Cool 
  - Cold
- peripheral pulse
  - normal 
  - increased 
  - reduced 
  - absent
- mucous membranes
  - possible values:
    - normal pink 
    - bright pink 
    - pale pink 
    - pale cyanotic 
    - bright red / injected 
    - dark cyanotic
  - 1 and 2 probably indicate a normal or slightly increased circulation 
  - 3 may occur in early shock 
  - 4 and 6 are indicative of serious circulatory compromise 
  - 5 is more indicative of a septicemia
- capillary refill time 
  - a clinical judgement. The longer the refill, the poorer the circulation
  - possible values: 
    - more than 3 sec
    - 3 sec exactly
    - less than 3 sec
- pain - a subjective judgement of the horse's pain level
  - possible values: 
    - alert, no pain 
    - depressed 
    - intermittent mild pain 
    - intermittent severe pain 
    - continuous severe pain
  - should NOT be treated as a ordered or discrete variable
  - In general, the more painful, the more likely it is to require surgery
  - prior treatment of pain may mask the pain level to some extent
- peristalsis
  - an indication of the activity in the horse's gut. As the gut becomes more distended or the horse becomes more toxic, the activity decreases 
  - possible values:
    - hypermotile 
    - normal 
    - hypomotile 
    - absent
    - distend_small
- abdominal distension
  - An IMPORTANT parameter
  - an animal with abdominal distension is likely to be painful and have reduced gut motility
  - a horse with severe abdominal distension is likely to require surgery just tio relieve the pressure
  - possible values:
    - none 
    - slight 
    - moderate 
    - severe
- nasogastric tube
  - this refers to any gas coming out of the tube
  - a large gas cap in the stomach is likely to give the horse discomfort
    - none 
    - slight 
    - significant
- nasogastric reflux
  - the greater amount of reflux, the more likelihood that there is some serious obstruction to the fluid passage from the rest of the intestine 
  - possible values:
    - none
    - more than 1 liter
    - less than 1 liter
- nasogastric reflux PH
  - linear
  - scale is from 0 to 14 with 7 being neutral
  - normal values are in the 3 to 4 range
- rectal examination - feces
  - absent feces probably indicates an obstruction
  - possible values:
    - normal 
    - increased 
    - decreased 
    - absent
- abdomen
  - possible values:
    - normal 
    - other 
    - firm feces in the large intestine 
    - distended small intestine 
    - distended large intestine
  - 3 is probably an obstruction caused by a mechanical impaction and is normally treated medically 
  - 4 and 5 indicate a surgical lesion 
- packed cell volume
  - linear
  - the # of red cells by volume in the blood
  - normal range is 30 to 50. The level rises as the circulation becomes compromised or as the animal becomes dehydrated. 
- total protein
  - linear
  - normal values lie in the 6-7.5 (gms/dL) range 
  - the higher the value the greater the dehydration 
- abdominocentesis appearance (abdomo_appearance)
  - a needle is put in the horse's abdomen and fluid is obtained from the abdominal cavity
  - normal fluid is clear while cloudy or serosanguinous indicates a compromised gut 
  - possible values: 
    - clear 
    - cloudy 
    - serosanguinous 
- abdomcentesis total protein
  - linear
  - the higher the level of protein the more likely it is to have a compromised gut. Values are in gms/dL 
- surgical lesion
  - possible values: 
    - Yes 
    - No
- leison types:
  - first number is site of lesion 
    -  1 = gastric 
    -  2 = sm intestine 
    -  3 = lg colon 
    -  4 = lg colon and cecum 
    -  5 = cecum 
    -  6 = transverse colon 
    -  7 = retum/descending colon 
    -  8 = uterus
    -  9 = bladder
    -  11 = all intestinal sites 
    -  00 = none
  - second number is type 
    -  1 = simple 
    -  2 = strangulation 
    -  3 = inflammation 
    -  4 = other
  - third number is subtype 
    - 1 = mechanical 
    - 2 = paralytic 
    - 0 = n/a 
  - fourth number is specific code 
    - 1 = obturation 
    - 2 = intrinsic
    - 3 = extrinsic
    - 4 = adynamic
    - 5 = volvulus/torsion
    - 6 = intussuption 
    - 7 = thromboembolic
    - 8 = hernia
    - 9 = lipoma/slenic incarceration 
    - 10 = displacement 
    - 0 = n/a
    - 28: cp_data
- is pathology data present for this case? (cp_data) 
  - Yes 
  - No 
- outcome
  - what eventually happened to the horse? 
  - possible values:
    - lived 
    - died 
    - was euthanized

In [83]:
# признаки и какие значения принимают 
# там где стоит комментарий - значит там значение, которого не должно быть согласно документации
# то есть в признаке pain встречается значение slight 1 раз, чего быть не должно
categories = {
    'surgery': ['yes', 'no'],
    'age': ['adult', 'young'],
    'temp_of_extremities': ['Normal', 'Warm', 'Cool', 'Cold'],
    'peripheral_pulse': ['normal', 'increased', 'reduced', 'absent'],
    'mucous_membrane': ['normal_pink', 'bright_pink', 'pale_pink', 'pale_cyanotic', 'bright_red', 'dark_cyanotic'],
    'capillary_refill_time': ['3', 'less_3_sec', 'more_3_sec'],
    'pain': ['depressed', 'mild_pain', 'alert', 'severe_pain', 'extreme_pain'],     # slight - 1 значение
    'peristalsis': ['hypermotile', 'normal', 'hypomotile', 'absent'],               # distend_small - 1 значение
    'abdominal_distention': ['none', 'slight', 'moderate', 'severe'],
    'nasogastric_tube': ['none', 'slight', 'significant'],
    'nasogastric_reflux': ['more_1_liter', 'none', 'less_1_liter'],                 # slight - 1 значение
    'rectal_exam_feces': ['normal', 'increased', 'decreased', 'absent'],            # serosanguious - 1 значение
    'abdomen': ['normal', 'other', 'firm', 'distend_small', 'distend_large'],
    'abdomo_appearance': ['clear', 'cloudy', 'serosanguious'],
    'surgical_lesion': ['yes', 'no'],
    'cp_data': ['yes', 'no'],
    'outcome': ['lived', 'died', 'was euthanized']
}

# признаки которые буду кодировать
categorical_columns = ['surgery', 'age', 'temp_of_extremities', 'peripheral_pulse', 'mucous_membrane', 
                       'capillary_refill_time', 'pain', 'peristalsis', 'abdominal_distention', 'nasogastric_tube',
                       'nasogastric_reflux', 'rectal_exam_feces', 'abdomen', 'abdomo_appearance', 'surgical_lesion',
                       'cp_data']

In [84]:
# делаю маску ошибочных строк, чтобы потом их удалить
mask = (hourse_data['pain'] == 'slight') |\
       (hourse_data['peristalsis'] == 'distend_small') |\
       (hourse_data['nasogastric_reflux'] == 'slight') |\
       (hourse_data['rectal_exam_feces'] == 'serosanguious')


print(hourse_data.shape)
# удаляю строки, где индексы соответствуют маске
row_df = hourse_data.drop(index=hourse_data[mask].index)
# пока что удаляю эти столбцы, так как их сложно закодировать
row_df = row_df.drop(columns=['lesion_1', 'lesion_2', 'lesion_3'])
print(row_df.shape)


print(row_df.shape)
# one hot encoding на все категориальные признаки
row_df = pd.get_dummies(row_df, columns=categorical_columns, dtype='int64')
print(row_df.shape)

(1235, 28)
(1231, 25)
(1231, 25)
(1231, 65)


In [85]:
row_df

,hospital_number,rectal_temp,pulse,respiratory_rate,nasogastric_reflux_ph,packed_cell_volume,total_protein,abdomo_protein,outcome,surgery_no,surgery_yes,age_adult,age_young,temp_of_extremities_cold,temp_of_extremities_cool,temp_of_extremities_normal,temp_of_extremities_warm,peripheral_pulse_absent,peripheral_pulse_increased,peripheral_pulse_normal,peripheral_pulse_reduced,mucous_membrane_bright_pink,mucous_membrane_bright_red,mucous_membrane_dark_cyanotic,mucous_membrane_normal_pink,mucous_membrane_pale_cyanotic,mucous_membrane_pale_pink,capillary_refill_time_3,capillary_refill_time_less_3_sec,capillary_refill_time_more_3_sec,pain_alert,pain_depressed,pain_extreme_pain,pain_mild_pain,pain_severe_pain,peristalsis_absent,peristalsis_hypermotile,peristalsis_hypomotile,peristalsis_normal,abdominal_distention_moderate,abdominal_distention_none,abdominal_distention_severe,abdominal_distention_slight,nasogastric_tube_none,nasogastric_tube_significant,nasogastric_tube_slight,nasogastric_reflux_less_1_liter,nasogastric_reflux_more_1_liter,nasogastric_reflux_none,rectal_exam_feces_absent,rectal_exam_feces_decreased,rectal_exam_feces_increased,rectal_exam_feces_normal,abdomen_distend_large,abdomen_distend_small,abdomen_firm,abdomen_normal,abdomen_other,abdomo_appearance_clear,abdomo_appearance_cloudy,abdomo_appearance_serosanguious,surgical_lesion_no,surgical_lesion_yes,cp_data_no,cp_data_yes
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,530001,38.1,132.0,24.0,6.5,57.0,8.5,3.4,died,0,1,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,1,0
1,533836,37.5,88.0,12.0,2.0,33.0,64.0,2.0,euthanized,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,1,0
2,529812,38.3,120.0,28.0,3.5,37.0,6.4,3.4,lived,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,1,0
3,5262541,37.1,72.0,30.0,2.0,53.0,7.0,3.9,lived,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,1
4,5299629,38.0,52.0,48.0,7.0,47.0,7.3,2.6,lived,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230,535246,38.5,129.0,48.0,2.0,57.0,66.0,2.0,lived,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,1,0
1231,528570,37.5,60.0,50.0,3.0,35.0,6.4,3.6,died,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,1
1232,529685,37.5,84.0,40.0,3.0,40.0,5.9,7.0,lived,0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1


In [92]:
from sklearn.model_selection import train_test_split, GridSearchCV
import xgboost as xgb
from sklearn.metrics import balanced_accuracy_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder

In [96]:
le = LabelEncoder()
y = le.fit_transform(row_df['outcome'])
X = row_df.drop(columns='outcome')

In [100]:
le_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
le_mapping 

{'died': 0, 'euthanized': 1, 'lived': 2}

In [101]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [108]:
xgb_clf = xgb.XGBClassifier(objective='multi:softmax', num_class=3)
parameters = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [30, 50, 100]
}
grid_search = GridSearchCV(estimator=xgb_clf, param_grid=parameters, scoring='accuracy', cv=5, verbose=10)

In [109]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV 1/5; 1/36] START learning_rate=0.1, max_depth=3, n_estimators=30............
[CV 1/5; 1/36] END learning_rate=0.1, max_depth=3, n_estimators=30;, score=0.726 total time=   0.0s
[CV 2/5; 1/36] START learning_rate=0.1, max_depth=3, n_estimators=30............
[CV 2/5; 1/36] END learning_rate=0.1, max_depth=3, n_estimators=30;, score=0.655 total time=   0.0s
[CV 3/5; 1/36] START learning_rate=0.1, max_depth=3, n_estimators=30............
[CV 3/5; 1/36] END learning_rate=0.1, max_depth=3, n_estimators=30;, score=0.756 total time=   0.0s
[CV 4/5; 1/36] START learning_rate=0.1, max_depth=3, n_estimators=30............
[CV 4/5; 1/36] END learning_rate=0.1, max_depth=3, n_estimators=30;, score=0.660 total time=   0.0s
[CV 5/5; 1/36] START learning_rate=0.1, max_depth=3, n_estimators=30............
[CV 5/5; 1/36] END learning_rate=0.1, max_depth=3, n_estimators=30;, score=0.760 total time=   0.0s
[CV 1/5; 2/36] START learning_rat

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_class=3,
                                     num_parallel_tree=None, ...),
             param_grid={'learning_rate': [0.1, 0.01, 0.001],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [30, 50, 100, 200]},
             scoring='accuracy', verbose=10)

In [110]:
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print("Лучшие параметры:", best_params)
print("Точность на обучающем наборе:", best_score)

y_pred = grid_search.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Точность на тестовом наборе:", accuracy)

Лучшие параметры: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
Точность на обучающем наборе: 0.7256137988190201
Точность на тестовом наборе: 0.6761133603238867
